밑의 코드 bs4(beautiful soup 4)를 이용해 3월 1~30일 까지의 기온을 수집할 수 있는 코드 이다. 이를 활용하여 CSV파일을 손쉽게 만들 수 있다.

In [28]:
#Logistic Regression을 위한 import
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt

# seed 고정
torch.manual_seed(1)

bs4는 웹 크롤링을 도우나 본 과제에서는 웹 스크랩을 하기 위한 도구로 사용함에 유의

In [2]:
#bs4를 위한 import
from bs4 import BeautifulSoup

In [29]:
#웹 스크랩을 하기 위한 import
import urllib.request as req
import re
#url의 날짜부분을 바꾸기 휘한 import
import datetime

In [30]:
#자료 수집을 시작하기 위한 날자의 설정
now = datetime.datetime(2021,1,1)
#날짜를 출력하는 형태의 설정(ex2022.04.24)
formattedDate = now.strftime("%Y.%m.%d")
#자료수집을 365일을 실행
n=0
for n in range(365):
  #자료를 가져올 주소의 설정, 문자열의 변수를 넣기위해 .format 사용
  url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do?auto_man=m&stn=0&dtm=&type=t99&reg=109&tm={}.19%3A00'.format(formattedDate)
  #설정된 url에서 자료를 가져오고
  res = req.urlopen(url)
  #이를 html의 자료들로 설정
  soup = BeautifulSoup(res,'html.parser')  

  #자료가 있는 table의 위치를 웹페이지의 console에서 찾아내고
  #이를 각각의 변수들에 입력
  if now.month<5 or now.month>9:
    Temperatures=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(6)')
    wet=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(11)')
    wind=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(13)')
    numbers = re.findall(r'\d+', wind.text)
    pressure=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(14)')
    rain=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(9)')
  else:
    Temperatures=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(6)')
    wet=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(10)')
    wind=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(12)')
    numbers = re.findall(r'\d+', wind.text)
    pressure=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(13)')
    rain=soup.select_one('#weather_table > tbody > tr:nth-of-type(4) > td:nth-of-type(9)')

  #위의 변수들을 출력
  print(Temperatures.text,',',wet.text[0],',',numbers[0]+'.'+numbers[1],',',pressure.text,',',rain.text.rjust(2,"0"))

  #위의 사항을 실했했을 경우 날짜가 1일이 지나도록 변경
  now += datetime.timedelta(days=1)
  formattedDate = now.strftime("%Y.%m.%d")

-1.5 , 7 , 2.4 , 1024.5 , 0 
-3.8 , 5 , 2.8 , 1031.1 , 0 
-4.4 , 4 , 1.1 , 1033.6 , 0 
-0.9 , 4 , 1.8 , 1028.8 , 0 
-6.5 , 3 , 1.9 , 1030.5 , 0.0
-4.2 , 6 , 3.4 , 1019.3 , 0.0
-14.2 , 4 , 2.1 , 1027.6 , 0 
-13.1 , 4 , 3.4 , 1024.1 , 0 
-10.2 , 4 , 4.2 , 1025.3 , 0 
-4.4 , 4 , 0.9 , 1028.1 , 0 
-5.4 , 5 , 1.3 , 1024.4 , 0 
-1.3 , 9 , 2.2 , 1015.7 , 2.5
5.8 , 8 , 2.1 , 1014.4 , 0.0
3.9 , 3 , 2.7 , 1020.7 , 0 
4.0 , 8 , 3.7 , 1016.2 , 0.2
-4.7 , 5 , 4.9 , 1026.9 , 0 
-2.8 , 6 , 1.9 , 1024.9 , 0.0
-4.9 , 4 , 4.8 , 1024.5 , 0.3


KeyboardInterrupt: ignored

In [27]:
#Logistic 함수의 설정
class Logistic(nn.Module):

  def __init__(self,num_inputs, num_outputs):
    super().__init__()
    self.linear = nn.Linear(num_inputs, num_outputs)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [26]:
dataset = np.loadtxt(
    '/content/drive/MyDrive/Colab Notebooks/data_210101_201231.csv', 
    delimiter=',', 
    dtype=np.float32)

#X0는 온도, X1는 습도, X2는 풍속, X3는 기압을 의미
x_train = torch.FloatTensor(dataset[:,:-1])
#기압이 다른 자료에 비해 값이 커 오차가 비약적으로 상승하기에 단위를 kPa로 변경
x_train[:,3]=x_train[:,3]/1000
#y는 강우량을 의미
y_train = torch.FloatTensor(dataset[:,[-1]])
#강우의 유무만 결정하기에 비가 안온다면 0을, 비가 온다면 1을 반환하도록 설정
i=0
for i in range(365):
  if y_train[i][0] > 0:
   y_train[i][0] = 1
  i+=1 

#input은 4 output은 1
model = Logistic(4,1)

#cost가 스윙하지 않는 lr값을 설정
optimizer = opt.SGD(model.parameters(),lr=0.01)

#accuray가 최고점에 있을때의 epoch 설정 
for epoch in range(7500):

  y_hypo = model(x_train)
  cost = func.binary_cross_entropy(y_hypo, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 9:
    
    prediction = y_hypo >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item()/len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, 7500, cost.item(), accuracy * 100,
    ))

Epoch    9/7500 Cost: 0.676850 Accuracy 67.95%
Epoch   19/7500 Cost: 0.667621 Accuracy 67.67%
Epoch   29/7500 Cost: 0.659862 Accuracy 67.67%
Epoch   39/7500 Cost: 0.653084 Accuracy 67.40%
Epoch   49/7500 Cost: 0.647001 Accuracy 67.67%
Epoch   59/7500 Cost: 0.641439 Accuracy 67.67%
Epoch   69/7500 Cost: 0.636296 Accuracy 67.95%
Epoch   79/7500 Cost: 0.631505 Accuracy 67.95%
Epoch   89/7500 Cost: 0.627023 Accuracy 68.22%
Epoch   99/7500 Cost: 0.622819 Accuracy 67.95%
Epoch  109/7500 Cost: 0.618867 Accuracy 67.67%
Epoch  119/7500 Cost: 0.615149 Accuracy 67.95%
Epoch  129/7500 Cost: 0.611646 Accuracy 67.67%
Epoch  139/7500 Cost: 0.608344 Accuracy 68.77%
Epoch  149/7500 Cost: 0.605229 Accuracy 68.49%
Epoch  159/7500 Cost: 0.602287 Accuracy 68.49%
Epoch  169/7500 Cost: 0.599508 Accuracy 67.95%
Epoch  179/7500 Cost: 0.596879 Accuracy 67.95%
Epoch  189/7500 Cost: 0.594392 Accuracy 67.40%
Epoch  199/7500 Cost: 0.592036 Accuracy 67.67%
Epoch  209/7500 Cost: 0.589802 Accuracy 68.22%
Epoch  219/75

In [56]:
#verify
#기압의 단위가 kPa 임에 주의
x=torch.FloatTensor([1.5,9,2.1,1.0134])
y=model(x)
#비가 올 확률
print(y)
#0.5를 기준으로 하여 결과를 예측
print(torch.round(y))

tensor([0.8280], grad_fn=<SigmoidBackward0>)
tensor([1.], grad_fn=<RoundBackward0>)


본 예측기의 장점 자료 수집기를 만들어서 사용했기에 경우에 따라선 더 많은 자료를 수집에 모델의 정확도를 올릴 수 있다.  
또한 그 자료들이 공신력 있는 웹페이지에서 가져왔기에 자료의 정확도 또한 높으며, 강수확률 예보법 이라는 기상 예측법에 사용하는 변수들을 이용해 모델을 만들었기에 변수와 결과의 관계가 시사하는 바가 큼.  
모델의 정확도가 80%를 넘기는 다소 우수한 성능.  
단점 이 모델은 강설량은 포함하지 않은 모델이기에 강수예측모델이 아니라 강우예측 모델임.  
기상청 웹페이지의 주소나 형태가 바뀔경우 자료수집에 애로가 생김.  
웹페이지 나타내는 정보의 특성상 일평균 온도,습도,풍속,기압이 아니라 19시의 변수들을 사용해야만 했음. 따라서 결과는 정확히 하자면 금일 강우의 유무를 판단하는 것이 아니라 금일 19시에 강우의 유무를 판단함.
